In [ ]:
import os
import glob
import pandas as pd
from google.colab import drive

# 0) 드라이브 마운트
drive.mount("/content/drive")

# 1) '30만 데이터' 폴더 경로 (여기만 맞춰)
base_dir = "/content/drive/MyDrive/30만 데이터"

# 2) 결과 저장 폴더
output_dir = os.path.join(base_dir, "_extracted_5cols")
os.makedirs(output_dir, exist_ok=True)

# 3) 추출할 컬럼 5개 (고정)
required_cols = []
required_cols.append("발급회원번호")
required_cols.append("이용금액_신용_B0M")
required_cols.append("잔액_B0M")
required_cols.append("이용건수_신용_B0M")
required_cols.append("기준년월")

# 4) 폴더 안 CSV 전부 수집 (8개면 8개 잡힘)
csv_files = sorted(glob.glob(os.path.join(base_dir, "*.csv")))
print("찾은 CSV 개수:", len(csv_files))
print("저장 폴더:", output_dir)

# 5) 인코딩 + 헤더(컬럼) 안전하게 읽기
def get_cols_and_encoding(path):
    enc_list = []
    enc_list.append("utf-8-sig")
    enc_list.append("utf-8")
    enc_list.append("cp949")
    enc_list.append("euc-kr")

    for enc in enc_list:
        try:
            tmp = pd.read_csv(path, nrows=0, encoding=enc)
            return tmp.columns.tolist(), enc
        except:
            pass

    return None, None

# 6) 파일별로 추출 저장 (절대 중단 X, NaN 생성 X)
chunksize = 200000

ok_files = 0
skip_files = 0

for csv_path in csv_files:
    try:
        file_name = os.path.basename(csv_path)

        cols, enc = get_cols_and_encoding(csv_path)
        if cols is None:
            print("❌ 헤더/인코딩 읽기 실패(스킵):", file_name)
            skip_files = skip_files + 1
            continue

        missing = []
        for c in required_cols:
            if c not in cols:
                missing.append(c)

        if len(missing) > 0:
            print("⚠ 컬럼 없음(스킵):", file_name, "| missing:", missing)
            skip_files = skip_files + 1
            continue

        out_path = os.path.join(output_dir, file_name.replace(".csv", "_5cols.csv"))
        if os.path.exists(out_path):
            os.remove(out_path)

        first_write = True
        total_rows = 0

        for chunk in pd.read_csv(csv_path, usecols=required_cols, chunksize=chunksize, encoding=enc, low_memory=False):
            chunk = chunk[required_cols].copy()
            chunk.to_csv(out_path, index=False, encoding="utf-8-sig", mode="a", header=first_write)
            first_write = False
            total_rows = total_rows + len(chunk)

        print("✅ 완료:", file_name, "| rows:", total_rows)
        ok_files = ok_files + 1

    except Exception as e:
        print("❌ 처리 실패(스킵하고 계속):", os.path.basename(csv_path), "|", str(e))
        skip_files = skip_files + 1
        continue

print("=== 요약 ===")
print("성공 파일:", ok_files)
print("스킵 파일:", skip_files)
